In [28]:
import numpy as np
import pandas as pd
import wfdb
import ast

In [29]:
def load_raw_data(df,sampling_rate,path) :
    if sampling_rate == 100 :
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else :
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal,meta in data])
    return data

In [30]:
path = 'E:\DataScience\ptb_xl\\'
sampling_rate = 100

In [31]:
# load and convert annotation data
Y = pd.read_csv(path + 'ptbxl_database.csv',index_col='ecg_id')

Y.scp_codes = Y.scp_codes.apply(
    lambda x: ast.literal_eval(x)
)

In [8]:
# load raw signal data
X = load_raw_data(df=Y,sampling_rate=sampling_rate,path=path)

In [35]:
# load scp_statements.csv ofr diagnostic aggregation
agg_df = pd.read_csv(path + 'scp_statements.csv',index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
print(agg_df.head())

                          description  diagnostic  form  rhythm  \
NDT    non-diagnostic T abnormalities         1.0   1.0     NaN   
NST_          non-specific ST changes         1.0   1.0     NaN   
DIG                  digitalis-effect         1.0   1.0     NaN   
LNGQT                long QT-interval         1.0   1.0     NaN   
NORM                       normal ECG         1.0   NaN     NaN   

      diagnostic_class diagnostic_subclass  \
NDT               STTC                STTC   
NST_              STTC                NST_   
DIG               STTC                STTC   
LNGQT             STTC                STTC   
NORM              NORM                NORM   

                                      Statement Category  \
NDT                    other ST-T descriptive statements   
NST_   Basic roots for coding ST-T changes and abnorm...   
DIG                    other ST-T descriptive statements   
LNGQT                  other ST-T descriptive statements   
NORM                

In [37]:
def aggregate_diagnostic(y_dic:pd.DataFrame) :
    tmp = []
    for key in y_dic.keys() :
        if key in agg_df.index :
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [38]:
# apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [39]:
# split into train/test
test_fold = 10

In [40]:
# train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[Y.strat_fold != test_fold].diagnostic_superclass

In [41]:
# test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [42]:
y_train.shape

(19634,)

In [53]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

one_hot = pd.DataFrame(mlb.fit_transform(y_train),
                          columns=mlb.classes_)
one_hot

,CD,HYP,MI,NORM,STTC
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
19629,0,0,0,0,1
19630,0,0,0,1,0
19631,0,0,0,0,1
19632,0,0,0,1,0


In [36]:
scp = pd.read_csv(f'{path}/scp_statements.csv',index_col=0)
scp.index

Index(['NDT', 'NST_', 'DIG', 'LNGQT', 'NORM', 'IMI', 'ASMI', 'LVH', 'LAFB',
       'ISC_', 'IRBBB', '1AVB', 'IVCD', 'ISCAL', 'CRBBB', 'CLBBB', 'ILMI',
       'LAO/LAE', 'AMI', 'ALMI', 'ISCIN', 'INJAS', 'LMI', 'ISCIL', 'LPFB',
       'ISCAS', 'INJAL', 'ISCLA', 'RVH', 'ANEUR', 'RAO/RAE', 'EL', 'WPW',
       'ILBBB', 'IPLMI', 'ISCAN', 'IPMI', 'SEHYP', 'INJIN', 'INJLA', 'PMI',
       '3AVB', 'INJIL', '2AVB', 'ABQRS', 'PVC', 'STD_', 'VCLVH', 'QWAVE',
       'LOWT', 'NT_', 'PAC', 'LPR', 'INVT', 'LVOLT', 'HVOLT', 'TAB_', 'STE_',
       'PRC(S)', 'SR', 'AFIB', 'STACH', 'SARRH', 'SBRAD', 'PACE', 'SVARR',
       'BIGU', 'AFLT', 'SVTAC', 'PSVT', 'TRIGU'],
      dtype='object')